In [1]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmocean as cmo
import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy
import sys,os

sys.path.append('../src')
import utils as ut

In [2]:
runn = 'tot2'
jmax = 55
olev = 36

In [3]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()

fnames = sorted(glob.glob(f'../data/ecefiles/n011/n011*.nc'))

ds = xr.open_dataset(fnames[0])
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
levmid = ds['olevel'].values
lev = ds['olevel_bounds'].values
time_bnds = ds['time_centered_bounds']
thick = ds['e3t'].values #Quasi-time-independent, treated as fixed

ds.close()
secs = (time_bnds[:12,1]-time_bnds[:12,0]).values / np.timedelta64(1, 's')

months = np.arange(0,12)

In [3]:
def get_fnames(run):
    fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*T.nc'))
    if run == 'n011':
        fnames = fnames[38:] #Omit the first 100 years which is spin up
        rrun = 'ctrl'
    else:
        rrun = run

    nmonths = 0
    for fname in fnames:
        ds = xr.open_dataset(fname)
        nmonths += len(ds.time_counter)
        ds.close()
    nyears = int(np.floor(nmonths/12))
    print(rrun,nmonths, 'months =',nyears,'y',int(nmonths-12*nyears),'m')
    return fnames,nmonths,nyears,rrun

In [4]:
ds = xr.open_dataset(f'../data/basinmask.nc').isel(y=slice(0,jmax),basin=slice(0,5))
lon = ds.lon
lat = ds.lat
mask = ds.mask
basin = ds.basin.values
ds.close()
mask = np.nansum(mask,axis=1)

In [5]:
runs = np.append(['n011'],runn)

ds = xr.open_dataset(f'../data/temperature_mon_ctrl.nc').isel(basin=slice(0,5))
tempc = ds.temp
ds.close()

ds = xr.open_dataset(f'../data/temperature_mon_{runn}.nc').isel(basin=slice(0,5))
time = ds.time.values/12
tempr = ds.temp
ds.close()

dtemp = tempr-tempc[:len(time)]

In [6]:
fnames,nmonthsr,nyears,rrun = get_fnames(runn)
ds = xr.open_dataset(fnames[0]).isel(y=slice(0,jmax))
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
ds.close()

var = {}
for run in runs:
    fnames,nmonths,nyears,rrun = get_fnames(run)
    var[run] = np.zeros((nmonthsr,lon.shape[0],lon.shape[1]))
    c = -1
    for fname in fnames:
        ds = xr.open_dataset(fname).isel(y=slice(0,jmax))
        for t,tt in enumerate(ds['time_counter'].values):
            c+=1
            if c>=nmonthsr:
                break
            var[run][c,:,:] = ds['thetao'].isel(olevel=olev,time_counter=t)
        if c>=nmonthsr:
            break
        

tot2 390 months = 32 y 6 m
ctrl 2400 months = 200 y 0 m
tot2 390 months = 32 y 6 m


In [7]:
cmap = 'cmo.balance'
vmin = -1
vmax = 1

mpl.rcParams['figure.figsize'] = (15,5)
mpl.rcParams['figure.subplot.wspace'] = .25
mpl.rcParams['figure.subplot.hspace'] = .1

In [8]:
prefix = f'substemp_{run}_mon'

framerate = nmonthsr/60

for m in range(0,10):
#for m in range(nmonthsr):
    fig = plt.figure()
    
    ax = fig.add_subplot(131,projection=ccrs.SouthPolarStereo())
    ax.set_extent([-180, 180, -80, -60], crs=ccrs.PlateCarree())
    for b,bas in enumerate(basin):
        ax.scatter(lon,lat,15*mask[b,:,:],c=ut.bcol[bas],lw=0,transform=ccrs.PlateCarree())

    ax.add_feature(cartopy.feature.LAND.with_scale('10m'),facecolor='.7')
    ax.coastlines(linewidth=0.4, zorder=2, resolution='10m')
        
    ax.set_title('Five ocean regions')
        
    ax = fig.add_subplot(132,projection=ccrs.SouthPolarStereo())
    ax.set_extent([-180, 180, -80, -60], crs=ccrs.PlateCarree())
    im = ax.scatter(lon,lat,15,c=var['tot2'][m,:,:]-var['n011'][m,:,:],cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
    #im = ax.pcolormesh(lon[:,1:-1],lat[:,1:-1],var['tot2'][m,:,1:-1]-var['n011'][m,:,1:-1],cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
    #im = ax.pcolormesh(lon,lat,var['tot2'][m,:,:]-var['n011'][m,:,:],cmap=cmap,vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())

    ax.add_feature(cartopy.feature.LAND.with_scale('10m'),facecolor='.7')
    ax.coastlines(linewidth=0.4, zorder=2, resolution='10m')
        
    axins = inset_axes(ax,width="5%",height="100%",loc='center right',borderpad=-2)
    cb = fig.colorbar(im, cax=axins)
    
    ax.set_title('Temperature at 370m depth [degC]')
    
    ax = fig.add_subplot(133)
    ax.axhline(0,0,1,color='.5',ls=':')
    for b,bas in enumerate(basin):
        ax.plot(time[:m+1],dtemp.sel(basin=bas)[:m+1],c=ut.bcol[bas])
        ax.scatter(time[m],dtemp.sel(basin=bas)[m],40,c=ut.bcol[bas])
    ax.set_xlim([0,np.ceil(nmonths/12)])
    ax.set_ylim([vmin,vmax]) 
    ax.set_yticklabels([])
    ax.set_title('Subsurface ocean temperature anomaly [degC]')
    ax.set_xlabel('Years')

    savename = f'../videos/{prefix}_{m:04.0f}.png'
    plt.savefig(savename,dpi=150)
    print('Saved',m,'of',nmonthsr,'months',end='\r')
    plt.close()
    
os.system(f'ffmpeg -y -r {framerate} -f image2 -s 1920x1080 -i ../videos/{prefix}_%04d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p ../videos/{prefix}.mp4')
os.system(f'rm -r ../videos/{prefix}*.png')

ffmpeg version 4.3.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wp,-D_FORTIFY_SOURCE=2 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld ' --extra-cflags=' -I/usr/include/rav1e' --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libvo-amrwbenc --enable-version3 --enable-bzlib --disable-crystalhd --enable-fontconfig --enable-frei0r --enable-gcrypt --enabl

0